In [1]:
import torch
from torch.utils.data import DataLoader
import numpy as np

from data_.modelnet10 import ModelNet10

import sys
sys.path.insert(0, './data_/')

CLASSES = {
    0: 'bathtub',
    1: 'chair',
    2: 'dresser',
    3: 'night_stand',
    4: 'sofa',
    5: 'toilet',
    6: 'bed',
    7: 'desk',
    8: 'monitor',
    9: 'table'
}
N_CLASSES = len(CLASSES)



train_dataset = ModelNet10(data_root='./data_/ModelNet10/', n_classes=N_CLASSES, idx2cls=CLASSES, split='train')

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)





{'bathtub': 0, 'chair': 1, 'dresser': 2, 'night_stand': 3, 'sofa': 4, 'toilet': 5, 'bed': 6, 'desk': 7, 'monitor': 8, 'table': 9}


In [2]:
import torchvision

In [3]:
for sample in train_dataloader:
    voxel, cls_idx = sample['voxel'], sample['cls_idx']
    print(voxel.shape)
    print(cls_idx.shape)
    print(cls_idx)
    v = voxel.reshape(32,32,32)
#     print(v.dtype)
#     print(v[v==1])
    from torchvision.utils import save_image


#     save_image(v[16].to(torch.int32), 'img1.png')
#     import matplotlib.pyplot as plt
#     plt.imshow(v[16]) #Needs to be in row,col order
#     plt.savefig('imge.png')
    
    from PIL import Image
#     Image.fromarray(my_array.reshape((50,50)).astype('uint8')*255)
    image = Image.fromarray(v[16].detach().numpy().astype('uint8')*255)
#     image = Image.fromarray(np.random.randint(low=0,high=2,size=(320,320)).astype('uint8')*255)
    image.save('img.png')
#     torchvision.io.write_png(v.to(torch.uint8), "img1.png")
    #     for row in v:
#         print(row)
    break

torch.Size([1, 1, 32, 32, 32])
torch.Size([1])
tensor([4])


In [4]:
from PIL import Image
#     Image.fromarray(my_array.reshape((50,50)).astype('uint8')*255)
for idx, im in enumerate(v.permute(1,2,0)):
    image = Image.fromarray(im.detach().numpy().astype('uint8')*255)
    #     image = Image.fromarray(np.random.randint(low=0,high=2,size=(320,320)).astype('uint8')*255)
    image.save('imgfolder/img'+str(idx)+'.png')

In [5]:
# commenets
# It is possible to convert Imagenet10 geometries into voxels and .png file and then
# make a .pwmb file and print them... but there are only 32 layers! not enoupg as they are 32*32*32 imgs

In [6]:
######################################################3

# junk test :

In [7]:
def int_tostring(num:int, desire_str_length:int):
    
    str_len = len(str(num))
    assert(str_len<=desire_str_length)
    if (str_len < desire_str_length):
        return '0'*(desire_str_length-str_len)+str(num)

In [6]:
from itertools import product

In [7]:
box_siz = 101 # should be an odd number

In [8]:
# [i for i in range(0-box_siz//2, 0+box_siz//2+1)]

In [17]:
def sphere(point):
    x, y, z = point
    r = box_siz//2
    return (x-box_siz//2)**2+(y-box_siz//2)**2+(z-box_siz//2)**2 - (r)**2

In [67]:
def gyroid(point):
    x, y, z = 2*np.pi*point[0]/box_siz, 2*np.pi*point[1]/box_siz, 2*np.pi*point[2]/box_siz
    x, y, z = x/2, y/2, z/2
#     r = box_siz//2
    return np.sin(x)*np.cos(y) + np.sin(y)*np.cos(z) + np.sin(z)*np.cos(x) #-0.5

In [68]:
def primitive(point):
    x, y, z = 2*np.pi*point[0]/box_siz, 2*np.pi*point[1]/box_siz, 2*np.pi*point[2]/box_siz
#     r = box_siz//2
    return np.cos(x) + np.cos(y) + np.cos(z) 

In [69]:
def sdf(point, func):
    return func(point)

In [70]:
sdf((0,0.5,0), sphere)

4950.25

In [37]:
sdf((0,0.5,2), gyroid)

0.3099164179565318

In [38]:
r_90_perc = box_siz*411/100
v_holder = np.zeros((box_siz,box_siz,box_siz))

for point in  product(range(0,box_siz), repeat=3):
#     print(point)
    sdf_value = sdf(point, sphere)
    if sdf_value<r_90_perc and sdf_value>-r_90_perc: 
#         print (point,sdf_value)
        v_holder[point] = 1

In [63]:
# v_holder.swapaxes(x,0,2)
v_holder = np.moveaxis(v_holder, -1, 0) # to make it : z*height*width

In [64]:
v_holder.shape

(5760, 3600)

In [65]:
int_tostring(104, 7)

'0000104'

In [60]:
v_holder[3]

array([0., 0., 0., ..., 0., 0., 0.])

In [51]:
from PIL import Image
for idx, im in enumerate(v_holder):
    image = Image.fromarray(im.astype('uint8')*255)
    #     image = Image.fromarray(np.random.randint(low=0,high=2,size=(320,320)).astype('uint8')*255)
    image.save('imgfolder/thumbnail'+int_tostring(idx, 5)+'.png')

In [71]:
from PIL import Image

# based on Anycube resolutions: width and heights are exchange... resolution and matrix difference
# Z      = 21
Width  = 3600
Height = 5760

r = Height//10


for z in range(-r,r):
    v_holder = np.zeros((Width, Height))
    print('z=',z,'/',2*r+1, '...')
    for width in range(-r, r):
        for height in range(-r, r):
            if (width**2 + height**2 + (z)**2- r**2>0):
                continue
#             if (width<Width//5 and height<Height//2)
            point = width+Width//2, height+Height//2, z+r
            sdf_value = sdf(point, gyroid)
            if sdf_value<0: 
            #         print (point,sdf_value)
                v_holder[point[0], point[1]] = 1
    
    image = Image.fromarray(v_holder.astype('uint8')*255)
    image.save('imgfolder/thumbnail'+int_tostring(z+r, 5)+'.png')   

z= -576 / 1153 ...
z= -575 / 1153 ...
z= -574 / 1153 ...
z= -573 / 1153 ...
z= -572 / 1153 ...
z= -571 / 1153 ...
z= -570 / 1153 ...
z= -569 / 1153 ...
z= -568 / 1153 ...
z= -567 / 1153 ...
z= -566 / 1153 ...
z= -565 / 1153 ...
z= -564 / 1153 ...
z= -563 / 1153 ...
z= -562 / 1153 ...
z= -561 / 1153 ...
z= -560 / 1153 ...
z= -559 / 1153 ...
z= -558 / 1153 ...
z= -557 / 1153 ...
z= -556 / 1153 ...
z= -555 / 1153 ...
z= -554 / 1153 ...
z= -553 / 1153 ...
z= -552 / 1153 ...
z= -551 / 1153 ...
z= -550 / 1153 ...
z= -549 / 1153 ...
z= -548 / 1153 ...
z= -547 / 1153 ...
z= -546 / 1153 ...
z= -545 / 1153 ...
z= -544 / 1153 ...
z= -543 / 1153 ...
z= -542 / 1153 ...
z= -541 / 1153 ...
z= -540 / 1153 ...
z= -539 / 1153 ...
z= -538 / 1153 ...
z= -537 / 1153 ...
z= -536 / 1153 ...
z= -535 / 1153 ...
z= -534 / 1153 ...
z= -533 / 1153 ...
z= -532 / 1153 ...
z= -531 / 1153 ...
z= -530 / 1153 ...
z= -529 / 1153 ...
z= -528 / 1153 ...
z= -527 / 1153 ...
z= -526 / 1153 ...
z= -525 / 1153 ...
z= -524 / 11

z= -144 / 1153 ...
z= -143 / 1153 ...
z= -142 / 1153 ...
z= -141 / 1153 ...
z= -140 / 1153 ...
z= -139 / 1153 ...
z= -138 / 1153 ...
z= -137 / 1153 ...
z= -136 / 1153 ...
z= -135 / 1153 ...
z= -134 / 1153 ...
z= -133 / 1153 ...
z= -132 / 1153 ...
z= -131 / 1153 ...
z= -130 / 1153 ...
z= -129 / 1153 ...
z= -128 / 1153 ...
z= -127 / 1153 ...
z= -126 / 1153 ...
z= -125 / 1153 ...
z= -124 / 1153 ...
z= -123 / 1153 ...
z= -122 / 1153 ...
z= -121 / 1153 ...
z= -120 / 1153 ...
z= -119 / 1153 ...
z= -118 / 1153 ...
z= -117 / 1153 ...
z= -116 / 1153 ...
z= -115 / 1153 ...
z= -114 / 1153 ...
z= -113 / 1153 ...
z= -112 / 1153 ...
z= -111 / 1153 ...
z= -110 / 1153 ...
z= -109 / 1153 ...
z= -108 / 1153 ...
z= -107 / 1153 ...
z= -106 / 1153 ...
z= -105 / 1153 ...
z= -104 / 1153 ...
z= -103 / 1153 ...
z= -102 / 1153 ...
z= -101 / 1153 ...
z= -100 / 1153 ...
z= -99 / 1153 ...
z= -98 / 1153 ...
z= -97 / 1153 ...
z= -96 / 1153 ...
z= -95 / 1153 ...
z= -94 / 1153 ...
z= -93 / 1153 ...
z= -92 / 1153 ...
z

z= 316 / 1153 ...
z= 317 / 1153 ...
z= 318 / 1153 ...
z= 319 / 1153 ...
z= 320 / 1153 ...
z= 321 / 1153 ...
z= 322 / 1153 ...
z= 323 / 1153 ...
z= 324 / 1153 ...
z= 325 / 1153 ...
z= 326 / 1153 ...
z= 327 / 1153 ...
z= 328 / 1153 ...
z= 329 / 1153 ...
z= 330 / 1153 ...
z= 331 / 1153 ...
z= 332 / 1153 ...
z= 333 / 1153 ...
z= 334 / 1153 ...
z= 335 / 1153 ...
z= 336 / 1153 ...
z= 337 / 1153 ...
z= 338 / 1153 ...
z= 339 / 1153 ...
z= 340 / 1153 ...
z= 341 / 1153 ...
z= 342 / 1153 ...
z= 343 / 1153 ...
z= 344 / 1153 ...
z= 345 / 1153 ...
z= 346 / 1153 ...
z= 347 / 1153 ...
z= 348 / 1153 ...
z= 349 / 1153 ...
z= 350 / 1153 ...
z= 351 / 1153 ...
z= 352 / 1153 ...
z= 353 / 1153 ...
z= 354 / 1153 ...
z= 355 / 1153 ...
z= 356 / 1153 ...
z= 357 / 1153 ...
z= 358 / 1153 ...
z= 359 / 1153 ...
z= 360 / 1153 ...
z= 361 / 1153 ...
z= 362 / 1153 ...
z= 363 / 1153 ...
z= 364 / 1153 ...
z= 365 / 1153 ...
z= 366 / 1153 ...
z= 367 / 1153 ...
z= 368 / 1153 ...
z= 369 / 1153 ...
z= 370 / 1153 ...
z= 371 / 1

In [ ]:
# Comments:
# - I have to select the same length for intger numbering of .png files: img001, img002, ...,img401 not img1, img2...img401
#      numbering shold start from 0 (if you have 401 (0 to 400) of .png files ---> in '.ini' file numFast = 401 
#      should be set.). 'numFast=' should be set to a numer that represents the number of layers (z axis), and 
#      display_pixels_x = 5760
#      display_pixels_y = 3600
#      these last two are Anycubic X 6k resolution configurations.


# - I have to use *555 as I shown above. It is only working with absolute 0 and 1.
# - When I put the .png files in a folder, then i need a 'config.ini' and 'pursasclicer.ini' files and a folder
#      to for 'thumbnail' that is used for showing on the printer screen. After putting all of these in a single file 
# - I must compress them into .zip file and then 'righ click+rename+.sls1'(remove .zip). the process of zipping 
#      should not be run on a parent folder. I must select .ini files and thumbnail folder and zip them and the rest (
#      it killed my time for hours to accidentally understand that...damnit).
# - I should now open the this .sl1s file in uvtools and convert it to .pwmb if I'm wokring on Anycube Mono X 6k. 
# - I need to define pngs with Width=5760, Height=3600 to be compatible with my printer (Anycub mono x 6K). otherwise
#      I have to modify display_pixels_x (Width) and display_pixels_y (Height).
#      I also need to 
# - You might run out of memery, if you consider a 3d np.ndarray for holding .pngs. the best practice is 
# to work on one .png and save it, then go to another layer and work on another .png file...